<a href="https://colab.research.google.com/github/darcor99/Python_Stock_analysis/blob/main/amd_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install alpaca-py
!pip install alpaca-trade-api
!pip install plotly
!pip install tensorflow
!pip install scikit-learn

In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [75]:
# Set all seeds
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [76]:
!curl -v https://paper-api.alpaca.markets/v2/account

*   Trying 35.194.67.18:443...
* Connected to paper-api.alpaca.markets (35.194.67.18) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
*  CAfile: /etc/ssl/certs/ca-certificates.crt
*  CApath: /etc/ssl/certs
* TLSv1.0 (OUT), TLS header, Certificate Status (22):
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* TLSv1.2 (IN), TLS header, Certificate Status (22):
* TLSv1.3 (IN), TLS handshake, Server hello (2):
* TLSv1.2 (IN), TLS header, Finished (20):
* TLSv1.2 (IN), TLS header, Supplemental data (23):
* TLSv1.3 (IN), TLS handshake, Encrypted Extensions (8):
* TLSv1.2 (IN), TLS header, Supplemental data (23):
* TLSv1.3 (IN), TLS handshake, Certificate (11):
* TLSv1.2 (IN), TLS header, Supplemental data (23):
* TLSv1.3 (IN), TLS handshake, CERT verify (15):
* TLSv1.2 (IN), TLS header, Supplemental data (23):
* TLSv1.3 (IN), TLS handshake, Finished (20):
* TLSv1.2 (OUT), TLS header, Finished (20):
* TLSv1.3 (OUT), TLS change cipher, Change cipher spec (1):
* TLSv1.2 (OUT), TLS

In [77]:
from google.colab import userdata
API_KEY = userdata.get('alpaca_api_key')
SECRET_KEY = userdata.get('alpaca_secret_key')

##setup trading client
trade_client = TradingClient(api_key=API_KEY, secret_key=SECRET_KEY, paper=True, url_override=None)

In [78]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3V1W5KBUPX',
    'accrued_fees': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': datetime.datetime(2025, 6, 2, 14, 13, 4, 702748, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': UUID('061448bc-80ed-4348-b5e7-ce177f03d92f'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'options_approved_level': 3,
    'options_buying_power': '100000',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled': T

In [79]:
##historical data client
client = StockHistoricalDataClient(API_KEY, SECRET_KEY)
client

In [80]:
from datetime import datetime
import datetime

symbol = "AMD" ###you can change this with any US company's ticker

request = StockBarsRequest(
    symbol_or_symbols=[symbol],
    timeframe=TimeFrame.Day,
    start = datetime.datetime(2021,1,1),
    end = datetime.datetime(2025,5,30)
)

bars = client.get_stock_bars(request).df
dataset = bars[bars.index.get_level_values(0) == symbol].copy()
dataset.reset_index(inplace=True, drop=True)
dataset.head()

,open,high,low,close,volume,trade_count,vwap
0,539.00,540.8000,515.085,522.86,4867450.0,116977.0,524.668643
1,521.55,526.7800,515.890,520.80,3383333.0,75278.0,521.251148
2,511.97,513.0998,499.500,500.49,5691529.0,139369.0,506.034319
3,508.28,516.4400,506.420,508.89,4126657.0,81401.0,510.021802
4,511.31,513.2400,504.510,510.40,3352930.0,76144.0,509.892575


In [81]:
def plot_stock_data(df):
  ##calculate SMAs (Simple moving averages)
  df['SMA5'] = df['close'].rolling(window=5).mean()
  df['SMA20'] = df['close'].rolling(window=20).mean()
  df['SMA50'] = df['close'].rolling(window=50).mean()

  fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
  vertical_spacing=0.03,
  subplot_titles=(f'{symbol} candlesticks with SMAs', 'volume'),
  row_width=[0.2, 0.7])

  #add candlesticks
  fig.add_trace(go.Candlestick(x=df.index,
  open=df['open'],
  high=df['high'],
  low=df['low'],
  close=df['close'],
  name='candlesticks'),
  row=1, col=1)

  ##add SMAs
  fig.add_trace(go.Scatter(x=df.index,
  y=df['SMA5'],
  name='SMA5'),
  row=1, col=1)

  fig.add_trace(go.Scatter(x=df.index,
  y=df['SMA20'],
  name='SMA20'),
  row=1, col=1)

  fig.add_trace(go.Scatter(x=df.index,
  y=df['SMA50'],
  name='SMA50'),
  row=1, col=1)

  fig.update_layout(title=f'{symbol} Price and volume analysis',
  xaxis_rangeslider_visible=False,
  yaxis_title = 'price (USD)',
  yaxis2_title = 'volume',
  height = 800)

  fig.show()

In [82]:
def add_indicators(df):
  ##calculate SMAs
  df['SMA5'] = df['close'].rolling(window=5).mean()
  df['SMA20'] = df['close'].rolling(window=20).mean()
  df['SMA50'] = df['close'].rolling(window=50).mean()

  df['Price_Change'] = df['close'].pct_change()

  #calc target - 1 if positive price difference, otherwise 0 for negative price difference
  df['target'] = np.where(df['Price_Change'] > 0.005, 1, 0) ##set target for when price increases 0.5 in a day

  #calc RSI relative strength index
  delta = df['close'].diff()
  gain = (delta.where(delta > 0, 0)).rolling(window=7).mean()
  loss = (-delta.where(delta < 0, 0)).rolling(window=7).mean()
  rs = gain / loss
  df['RSI'] = 100 - (100 / (1 + rs))

    # ✅ Lag features
  df['close_t-1'] = df['close'].shift(1)
  df['close_t-2'] = df['close'].shift(2)
  df['volume_t-1'] = df['volume'].shift(1)
  df['RSI_t-1'] = df['RSI'].shift(1)

  # ✅ Optional: momentum-type features
  df['momentum_3'] = df['close'] - df['close'].shift(3)
  df['momentum_5'] = df['close'] - df['close'].shift(5)
  df['price_to_vwap'] = df['close'] / df['vwap']

  return df

In [83]:
def prepare_data(df, look_back):
  features = ['close', 'SMA5', 'SMA20', 'SMA50', 'Price_Change', 'RSI', 'volume', 'close_t-1','close_t-2','volume_t-1','RSI_t-1', 'momentum_3', 'momentum_5', 'vwap', 'price_to_vwap']
  df = df.dropna()

  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(df[features])

  X,y = [],[]

  for i in range(look_back, len(scaled_data) -1):
    X.append(scaled_data[i-look_back:i])
    y.append(df['target'].iloc[i])

  return np.array(X), np.array(y)

In [84]:
def create_model(input_shape):
  model = Sequential([
        LSTM(128, input_shape=input_shape, return_sequences=True),
        Dropout(0.1),
        LSTM(64, return_sequences=False),
        Dropout(0.1),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

  optimizer = Adam(learning_rate=0.0005)

  model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
  return model

In [85]:
##visualise the data:
symbol = symbol
df = dataset.copy()
print('plotting stock data..')
plot_stock_data(df)


plotting stock data..


In [86]:
if __name__ == '__main__':

  symbol = symbol
  look_back = 10

  print('fetching stock data..')
  df = dataset.copy()

  #print('plotting stock data..')
  #plot_stock_data(df)

  print('adding indicators..')
  df = add_indicators(df)

  print('preparing data..')
  X,y = prepare_data(df, look_back)

  #split dataset into training and test data
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  print('training model..')
  model = create_model(input_shape = (look_back,*X.shape[2:]))

  training_score = model.evaluate(X_train, y_train, verbose=0)
  test_score = model.evaluate(X_test, y_test, verbose=0)

  print("model performance:")
  print("Training Accuracy: ", training_score[1])
  print("Testing Accuracy: ", test_score[1])


  last_sequence = X[-1:]
  tomorrow_prediction = model.predict(last_sequence)
  print("Tomorrow's prediction: ", tomorrow_prediction[0][0]) ##probability of price increase
  print(f"Model predicts {symbol} will decrease in value tomorrow") if tomorrow_prediction < 0.5 else print(f"Model predicts {symbol} will increase in value tomorrow")




fetching stock data..
adding indicators..
preparing data..
training model..


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



model performance:
Training Accuracy:  0.5095694065093994
Testing Accuracy:  0.5428571701049805
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
Tomorrow's prediction:  0.4630403
Model predicts NFLX will decrease in value tomorrow


In [87]:
print(training_score) ##[loss,accuracy ]

[0.6911019682884216, 0.5095694065093994]
